 # Tensorflow 数据格式处理大全
 Tensorflow入门难，难在整个计算过程中充斥着让人仅仅知道形式，而无法看到具体内容的图，比如tensor，之前只有在run的时候才能查看，非常不方便。TF提供了很多api，从读取目录到解析图片，只是通过tf框架读入的内容，往往都是tensor形式存储在内存中，很难查看，也无法得知操作是否正确，很难查找问题。
 
所幸新版本TF，提供了tf.enable_eager_execution()模式，可能极大方便我们实时观察tf中数据的变化，感觉有点借鉴了pytorch的思想。

本说明基于最新的tf 1.13版本，应该也兼容2.0版本。注意该版本需要将cuda更新到10.0，cuda最新版是10.1，注意不要装错了。

 首先导入需要的环境包

In [1]:
import tensorflow as tf
import numpy as np
tf.VERSION
tf.enable_eager_execution()

'1.13.1'

 ## 1. Tensor 与 numpy
 ### 1.1 基于两个框架的操作的转换
 TensorFlow operations 会自动将 NumPy ndarrays 转化为 Tensors。
 同样，Numpy operations 会自动将 Tensors 转化为 numpy ndarrays

In [2]:
ndarray = np.ones([3, 3])
tensor = tf.multiply(ndarray, 42)
print(tensor)
np_arr = np.add(tensor, 1)
print(np_arr)

tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]


 如果使用正常的加减乘除也是可以的。

In [3]:
print(ndarray + [[1, 1, 1], [1, 1, 1], [1, 1, 1]])
print(tensor + [[1, 1, 1], [1, 1, 1], [1, 1, 1]])
print(np_arr * 42)
print(tensor * 42)

[[2. 2. 2.]
 [2. 2. 2.]
 [2. 2. 2.]]
tf.Tensor(
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]], shape=(3, 3), dtype=float64)
[[1806. 1806. 1806.]
 [1806. 1806. 1806.]
 [1806. 1806. 1806.]]
tf.Tensor(
[[1764. 1764. 1764.]
 [1764. 1764. 1764.]
 [1764. 1764. 1764.]], shape=(3, 3), dtype=float64)


 ### 1.2 numpy -> tensor

In [4]:
print(tf.convert_to_tensor(np_arr))

tf.Tensor(
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]], shape=(3, 3), dtype=float64)


 ### 1.3 tensor -> numpy

In [5]:
print(tensor.numpy())

[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


## 2. Dataset与numpy & tensors
### 2.1 numpy(tensors) -> Dataset

In [6]:
ds_tensors1 = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])
ds_tensors2 = tf.data.Dataset.from_tensor_slices(np.ones([3,3]))
print(ds_tensors1)
print(ds_tensors2)

<DatasetV1Adapter shapes: (), types: tf.int32>
<DatasetV1Adapter shapes: (3,), types: tf.float64>


可以看到，Dataset在显示shape的时候，并不显示第一维，第一维默认是batch。
如果存在多个变量，可以采用下面的方式。

In [7]:
features = np.ones([100, 4, 4])
labels = np.ones([100, 1])
ds_tensor3 = tf.data.Dataset.from_tensor_slices((features, labels))
print(ds_tensor3)

<DatasetV1Adapter shapes: ((4, 4), (1,)), types: (tf.float64, tf.float64)>


这里，Dataset的大小变为(4, 4), (1, )
如果变量较多，我们还可以对其进行命名。

In [8]:
ds_tensor4 = tf.data.Dataset.from_tensor_slices({"a": features, "b": labels})
print(ds_tensor4)

<DatasetV1Adapter shapes: {a: (4, 4), b: (1,)}, types: {a: tf.float64, b: tf.float64}>


需要注意的是，Dataset还有一个函数from_tensors()，一般不用这个。因为这个函数不会将第一维视为batch。

In [9]:
ds_tensor5 = tf.data.Dataset.from_tensors({"a": features, "b": labels})
ds_tensor5.batch(4)
print(ds_tensor5)

<DatasetV1Adapter shapes: {a: (?, 100, 4, 4), b: (?, 100, 1)}, types: {a: tf.float64, b: tf.float64}>

<DatasetV1Adapter shapes: {a: (100, 4, 4), b: (100, 1)}, types: {a: tf.float64, b: tf.float64}>


  ### 2.2 Dataset -> Dataset
 Dataset合并到到Dataset跟numpy直接生成有一点点不一样

In [10]:
features_ds = tf.data.Dataset.from_tensor_slices(features)
label_ds = tf.data.Dataset.from_tensor_slices(labels)
ds_tensor6 = tf.data.Dataset.zip((features_ds, label_ds))
print(ds_tensor6)

<DatasetV1Adapter shapes: ((4, 4), (1,)), types: (tf.float64, tf.float64)>


 取Dataset的子集，当参数数量大于数据集大小时，返回整个dataset

In [11]:
ds_tensor6.take(10)

<DatasetV1Adapter shapes: ((4, 4), (1,)), types: (tf.float64, tf.float64)>

 ### 2.3 Dataset -> tensors & numpy array
 当一部分数据转换成Dataset之后，我们想观察一下这个数据集里有哪些，希望转换回来。
 Dataset标准做法是使用tf.data.Iterator，但是也可以使用python自己的迭代器实现，当且仅当eager execution开启的时候。

In [12]:
features = np.ones([10, 2, 5])
labels = np.ones([10, 1])
ds_tensor7 = tf.data.Dataset.from_tensor_slices((features, labels))
for f, l in ds_tensor7.take(2):
    #print(f, l) # 输出tensor
    print(f.numpy(), l.numpy())  # 输出numpy

Instructions for updating:
Colocations handled automatically by placer.
[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]] [1.]
[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]] [1.]


如果设置了batch，则会按照batch大小输出

In [13]:
ds_tensor8 = ds_tensor7.batch(4)
for f, l in ds_tensor8.take(2):
    #print(f, l)
    print(f.numpy(), l.numpy())

[[[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]

 [[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]

 [[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]

 [[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]] [[1.]
 [1.]
 [1.]
 [1.]]
[[[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]

 [[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]

 [[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]

 [[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]] [[1.]
 [1.]
 [1.]
 [1.]]


如果想取出整个数据集，可以将batch设置的比数据集更大的一个值

In [14]:
ds_tensor9 = ds_tensor7.batch(100, drop_remainder=False)
for f, l in ds_tensor9:
    #print(f, l)
    print(f.numpy(), l.numpy())

[[[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]

 [[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]

 [[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]

 [[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]

 [[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]

 [[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]

 [[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]

 [[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]

 [[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]

 [[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]] [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]


还有一种方法，使用python的迭代器

In [15]:
iteration = iter(ds_tensor7)
next(iteration)

(<tf.Tensor: id=83, shape=(2, 5), dtype=float64, numpy=
 array([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])>,
 <tf.Tensor: id=84, shape=(1,), dtype=float64, numpy=array([1.])>)

如果没有开启eager execution，使用起来会麻烦一些，分为单次迭代、可初始化、可重新初始化，以及可馈送。具体可参考https: // www.tensorflow.org/guide/datasets#creating_an_iterator
感觉用起来挺麻烦的，待总结需求情况。
下面是常规使用方法

dataset = tf.data.Dataset.range(5)

iterator = dataset.make_initializable_iterator()

next_element = iterator.get_next()

返回的next_element是tensor。既然2.0默认开启eager模式，这种方式就暂时不考虑了。

# 3. 更加复杂的Dataset构成
对于复杂的datset，需要先对每一个变量进行构建，然后再通过zip方式拼接在一起。

对于直接混合方式的数据，tf是无法处理的。例如，在list中存储一组dict，或者list中包含多个矩阵。

In [47]:
n_observations = int(1e2)

# multi input and output
img1 = np.arange(n_observations*25).reshape([n_observations, 5, 5]).astype(np.float64)
img2 = np.arange(n_observations*16).reshape([n_observations, 4, 4]).astype(np.float64)
img1 = tf.data.Dataset.from_tensor_slices(img1)
img2 = tf.data.Dataset.from_tensor_slices(img2)
img = tf.data.Dataset.zip((img1, img2))
img
label1 = ["cat"]*n_observations
label2 = [0]*n_observations
label1 = tf.data.Dataset.from_tensor_slices(label1)
label2 = tf.data.Dataset.from_tensor_slices(label2)
label = tf.data.Dataset.zip((label1, label2))
label
ds = tf.data.Dataset.zip((img, label))
ds

# 带名称的字典形式 multi input and output
img1 = np.arange(n_observations*25).reshape([n_observations, 5, 5]).astype(np.float64)
img2 = np.arange(n_observations*16).reshape([n_observations, 4, 4]).astype(np.float64)
img1 = tf.data.Dataset.from_tensor_slices({"im1": img1})
img2 = tf.data.Dataset.from_tensor_slices({"im2": img2})
img = tf.data.Dataset.zip((img1, img2))
img
label1 = ["cat"]*n_observations
label2 = [0]*n_observations
label1 = tf.data.Dataset.from_tensor_slices({"lb1":label1})
label2 = tf.data.Dataset.from_tensor_slices({"lb2":label2})
label = tf.data.Dataset.zip((label1, label2))
label
ds = tf.data.Dataset.zip((img, label))
ds
def change_format(x1, x2):
    return {"inputs":x1, "outputs":x2}
ds = ds.map(change_format)
ds
    

<DatasetV1Adapter shapes: ((5, 5), (4, 4)), types: (tf.float64, tf.float64)>

<DatasetV1Adapter shapes: ((), ()), types: (tf.string, tf.int32)>

<DatasetV1Adapter shapes: (((5, 5), (4, 4)), ((), ())), types: ((tf.float64, tf.float64), (tf.string, tf.int32))>

<DatasetV1Adapter shapes: ({im1: (5, 5)}, {im2: (4, 4)}), types: ({im1: tf.float64}, {im2: tf.float64})>

<DatasetV1Adapter shapes: ({lb1: ()}, {lb2: ()}), types: ({lb1: tf.string}, {lb2: tf.int32})>

<DatasetV1Adapter shapes: (({im1: (5, 5)}, {im2: (4, 4)}), ({lb1: ()}, {lb2: ()})), types: (({im1: tf.float64}, {im2: tf.float64}), ({lb1: tf.string}, {lb2: tf.int32}))>

<DatasetV1Adapter shapes: {inputs: ({im1: (5, 5)}, {im2: (4, 4)}), outputs: ({lb1: ()}, {lb2: ()})}, types: {inputs: ({im1: tf.float64}, {im2: tf.float64}), outputs: ({lb1: tf.string}, {lb2: tf.int32})}>

 ## 4. Dataset 与 TFRecord
 与TFRecord打交道，先把下面三个函数加上，分别处理字符、浮点型和整数型（包括bool型）。
 TFRecord之所以这样设计，是因为在数据读取时，如果每个数据都是一个连续序列的话，读取速度会快很多。

 创建要存储的数据
 the number of observations in the dataset

In [16]:
n_observations = int(1e2)

 ### 4.1 5x5 image float数据读写
 使用numpy生成5x5的float类型数据

In [17]:
feature1 = np.arange(n_observations*25).reshape([n_observations, 5, 5]).astype(np.float64)
feature1 = tf.data.Dataset.from_tensor_slices(feature1)
print(feature1)
#map函数的参数为一个函数，可以分别单独处理dataset中的每一个元素
restore_shape= feature1.output_shapes
feature1 = feature1.map(tf.serialize_tensor)
print(feature1)

<DatasetV1Adapter shapes: (5, 5), types: tf.float64>
<DatasetV1Adapter shapes: (), types: tf.string>


写入TFRecord文档中

In [18]:
filename = "test_float_matrix.tfrec"
tfrec = tf.data.experimental.TFRecordWriter(filename)
tfrec.write(feature1)

读取存储的文档

In [19]:
def parse(x):
    result = tf.parse_tensor(x, out_type=tf.float64)  # 注意查看存储时候的数据类型需要一致
    result = tf.reshape(result, tf.TensorShape([5,5])) # 不进行此操作，shape会显示为<unknown>，影响后续操作
    return result
ds2 = tf.data.TFRecordDataset(filename).map(parse)
ds2
for x in ds2.take(2):
    print(x)

<DatasetV1Adapter shapes: (5, 5), types: tf.float64>

tf.Tensor(
[[ 0.  1.  2.  3.  4.]
 [ 5.  6.  7.  8.  9.]
 [10. 11. 12. 13. 14.]
 [15. 16. 17. 18. 19.]
 [20. 21. 22. 23. 24.]], shape=(5, 5), dtype=float64)
tf.Tensor(
[[25. 26. 27. 28. 29.]
 [30. 31. 32. 33. 34.]
 [35. 36. 37. 38. 39.]
 [40. 41. 42. 43. 44.]
 [45. 46. 47. 48. 49.]], shape=(5, 5), dtype=float64)


### 4.2 字符串 string feature 数据读写

In [20]:
random_choice = np.random.randint(0, 5, n_observations).astype(np.int64)
strings = np.array([b'cat', b'dog', b'chicken', b'horse', b'goat'])
feature2 = strings[random_choice]
feature2 = tf.data.Dataset.from_tensor_slices(feature2)
print(feature2)

<DatasetV1Adapter shapes: (), types: tf.string>


In [21]:
filename = "test_string.tfrec"
tfrec = tf.data.experimental.TFRecordWriter(filename)
tfrec.write(feature2)

ds2 = tf.data.TFRecordDataset(filename)

for x in ds2.take(2):
    print(x)

tf.Tensor(b'chicken', shape=(), dtype=string)
tf.Tensor(b'cat', shape=(), dtype=string)


### 4.3 1x1维数字数据读写

In [39]:
random_choice = np.random.randint(0, 5, n_observations).astype(np.int64)
feature3 = tf.data.Dataset.from_tensor_slices(random_choice)
print(feature3)
feature3 = feature3.map(tf.serialize_tensor)
feature3

<DatasetV1Adapter shapes: (), types: tf.int64>


<DatasetV1Adapter shapes: (), types: tf.string>

In [42]:
filename = "test_int.tfrec"
tfrec = tf.data.experimental.TFRecordWriter(filename)
tfrec.write(feature3)
def parse(x):
    result = tf.parse_tensor(x, out_type=tf.int64)  # 注意查看存储时候的数据类型需要一致
    return result
ds3 = tf.data.TFRecordDataset(filename).map(parse)

for x in ds3.take(2):
    print(x)

tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)


### 4.4 多个特征同时写入
在写入多个特征时，通常用于数据和标签一一对应存储的情况，这时需要用到tf.example。注意，对于每一个包含多个项目的数据（下面统称为一个example），如果这个项目是单个值，可以使用下面的函数；如果是数组或矩阵，要先转化为字符串再进行存储。

In [22]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int32_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int32_list=tf.train.Int32List(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

然后定义字符串化每一个example的函数

In [23]:
# 单独每一个序列化
def serialize_example(feature1, feature2, feature3):
    """
    Creates a tf.Example message ready to be written to a file.
    """
    # Create a dictionary mapping the feature name to the tf.Example-compatible
    # data type.
    feature = {
        'feature1': _bytes_feature(feature1),
        'feature2': _bytes_feature(feature2),
        'feature3': _int64_feature(feature3),
    }
    # Create a Features message using tf.train.Example.
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

# example 序列化
def tf_serialize_example(f1,f2,f3):
    tf_string = tf.py_func( # 2.0好像会使用tf.py_function，但是在1.13版本中，如果替换成tf.py_function会出错
      serialize_example, 
      (f1,f2,f3),  # pass these args to the above function.
      tf.string)      # the return type is <a href="../../api_docs/python/tf#string"><code>tf.string</code></a>.
    return tf.reshape(tf_string, ())
feature3 = np.random.randint(0, 5, n_observations).astype(np.int64)
feature3 = tf.data.Dataset.from_tensor_slices(feature3)
multi_ds = tf.data.Dataset.zip((feature1, feature2, feature3))
print(multi_ds)
multi_ds2 = multi_ds.map(tf_serialize_example)
print(multi_ds2)

<DatasetV1Adapter shapes: ((), (), ()), types: (tf.string, tf.string, tf.int64)>
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    
<DatasetV1Adapter shapes: (), types: tf.string>


写入TFRecord文档

In [24]:
filename = 'test_multi_data.tfrecord'
writer = tf.data.experimental.TFRecordWriter(filename)
writer.write(multi_ds2)

读入TFRecord文档

In [25]:
filenames = [filename]
raw_dataset = tf.data.TFRecordDataset(filenames)
raw_dataset

<TFRecordDatasetV1 shapes: (), types: tf.string>

In [26]:
# 读入时需要知道原先都有哪些格式
feature_description = {
    'feature1': tf.FixedLenFeature([], tf.string, default_value=''),
    'feature2': tf.FixedLenFeature([], tf.string, default_value=''),
    'feature3': tf.FixedLenFeature([], tf.int64, default_value=0)
}
def _parse_function(example_proto):
    # Parse the input tf.Example proto using the dictionary above.
    return tf.parse_single_example(example_proto, feature_description)
parsed_dataset = raw_dataset.map(_parse_function)
parsed_dataset

<DatasetV1Adapter shapes: {feature1: (), feature2: (), feature3: ()}, types: {feature1: tf.string, feature2: tf.string, feature3: tf.int64}>

解析feature1的float矩阵

In [27]:
def _parse_feature1(dictf):
    res = tf.parse_tensor(dictf['feature1'], out_type=tf.float64)
    res = tf.reshape(res, tf.TensorShape([5,5]))
    return res,dictf['feature2'],dictf['feature3']
parsed_dataset2 = parsed_dataset.map(_parse_feature1)
parsed_dataset2
for a,b,c in parsed_dataset2.take(2):
    print(a)
    print(b)
    print(c)

<DatasetV1Adapter shapes: ((5, 5), (), ()), types: (tf.float64, tf.string, tf.int64)>

tf.Tensor(
[[ 0.  1.  2.  3.  4.]
 [ 5.  6.  7.  8.  9.]
 [10. 11. 12. 13. 14.]
 [15. 16. 17. 18. 19.]
 [20. 21. 22. 23. 24.]], shape=(5, 5), dtype=float64)
tf.Tensor(b'chicken', shape=(), dtype=string)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(
[[25. 26. 27. 28. 29.]
 [30. 31. 32. 33. 34.]
 [35. 36. 37. 38. 39.]
 [40. 41. 42. 43. 44.]
 [45. 46. 47. 48. 49.]], shape=(5, 5), dtype=float64)
tf.Tensor(b'cat', shape=(), dtype=string)
tf.Tensor(1, shape=(), dtype=int64)


## 5. 实际使用pipline形式
参考上面的内容，应当很容易看到官网教程中https://www.tensorflow.org/tutorials/load_data/images#build_a_tfdatadataset 的描述。
下面是伪代码：

已知有两个list分别包含，all_image_paths、all_label_paths,这个可通过外部维护

In [32]:
# all_image_paths=[]
# all_label_paths=[]
# def read_image(path):
#     # 返回读取的image
#     pass
# def read_label(path):
#     # 返回读取的label
#     pass
# def load_and_preprocess_from_path_label(img_path, label_path):
#       return read_image(img_path), read_label(label_path)

# ds = tf.data.Dataset.from_tensor_slices((all_image_paths, all_label_paths))#这个比较小，可以都读入内存
# AUTOTUNE = tf.data.experimental.AUTOTUNE
# ds = ds.map(load_and_preprocess_from_path_label, num_parallel_calls=AUTOTUNE)


具体开始训练

In [33]:
# # BATCH_SIZE = 32
# image_count = len(all_image_paths)
# # 将随机打乱的buffer设置到跟数据集一样大，这样能保证数据成分随机化
# ds = ds.shuffle(buffer_size=image_count)
# # 先打乱再repeat可以保证不同epoch之间数据不重复，这里repeat可以设置int参数，表示具体重复多少次；为空代表无限循环。
# ds = ds.repeat()
# ds = ds.batch(BATCH_SIZE)
# # `prefetch` 可以让dataset在训练过程中，在后台读取batch
# ds = ds.prefetch(buffer_size=AUTOTUNE)

在这个网址中，讲到了如何在keras中使用dataset https://tensorflow.google.cn/guide/keras#input_tfdata_datasets

在一般模型中，dataset迭代生成的结果本身就是tensor，可以直接输入到模型中。

### 4.1 Keras 模型示例

In [28]:
BATCH_SIZE = 32
IMG_NUM = 1000
x_train = np.random.normal(loc=0, scale=1, size=[IMG_NUM, 28, 28])
y_train = np.random.choice(10, size=IMG_NUM)
ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
AUTOTUNE = tf.data.experimental.AUTOTUNE
# 将随机打乱的buffer设置到跟数据集一样大，这样能保证数据成分随机化
ds = ds.shuffle(buffer_size=1000)
# 先打乱再repeat可以保证不同epoch之间数据不重复，这里repeat可以设置int参数，表示具体重复多少次；为空代表无限循环。
ds = ds.repeat()
ds = ds.batch(BATCH_SIZE)
# `prefetch` 可以让dataset在训练过程中，在后台读取batch
ds = ds.prefetch(buffer_size=AUTOTUNE)
ds

<DatasetV1Adapter shapes: ((?, 28, 28), (?,)), types: (tf.float64, tf.int32)>

简单模型

In [29]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.summary()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
steps_per_epoch=tf.ceil(IMG_NUM/BATCH_SIZE).numpy().astype(np.int32)
model.fit(ds, epochs=5, steps_per_epoch=steps_per_epoch)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
32/32 [==============================] - 3s 102ms/step - loss: 2.8052 - acc: 0.1104
Epoch 2/5
32/32 [==============================] - 0s 8ms/s

高级模型

In [30]:
inputs = tf.keras.Input(shape=(28,28))  # Returns a placeholder tensor

# A layer instance is callable on a tensor, and returns a tensor.
x = tf.keras.layers.Flatten()(inputs)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
predictions = tf.keras.layers.Dense(10, activation='softmax')(x)
model2 = tf.keras.Model(inputs=inputs, outputs=predictions)
model2.summary()

model2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
steps_per_epoch=tf.ceil(IMG_NUM/BATCH_SIZE).numpy().astype(np.int32)
model2.fit(ds, epochs=5, steps_per_epoch=steps_per_epoch)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
32/32 [==============================] - 1s 23ms/step - loss: 2.7633 - acc: 0.1064
Epoch 2/5
32/32 [==========================

### 4.2 Tensorflow 模型示例 非eager excution

In [31]:
# x_train = np.random.normal(loc=0, scale=1, size=[IMG_NUM, 28, 28])
# y_train = np.random.choice(10, size=IMG_NUM)
# y_train = tf.one_hot(y_train, 10)
# ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
# ds.batch(32)
# inputs = tf.placeholder(tf.float64, shape=[None, 28, 28])
# outputs = tf.placeholder(tf.int32, shape=[None, 10])
# def model3(inputs, outputs):
#     x = tf.layers.Flatten()(inputs)
#     x = tf.layers.Dense(512, activation='relu')(x)
#     x = tf.layers.Dropout(0.2)(x)
#     pred = tf.layers.Dense(10, activation='softmax')(x)
#     loss = tf.losses.softmax_cross_entropy(outputs, pred)
#     train = tf.train.AdamOptimizer().minimize(loss)
#     return pred, loss, train
# p, l, t = model3(inputs, outputs)


# sess = tf.Session()
# init = tf.global_variables_initializer()
# sess.run(init)

# iterator = ds.make_one_shot_iterator()
# next_element = iterator.get_next()

# for i in range(1000):
#     in_var, out_var = sess.run(next_element)
#     sess.run(t, feed_dict={inputs:in_var, outputs:out_var})
#     print(loss.numpy())

